In [1]:
import numpy as np
import os
from nltk.stem import PorterStemmer
import nltk
import enchant
import pickle

# PREPROCESSING

In [5]:
def preprocess(content,U,totalCount):
    d={}
    words=content.split()
    ps = PorterStemmer()
    engDict=enchant.Dict("en_US")
    for word in words:
        
        if (not word.isalnum()) and (word != '$'):
            continue
        if word.isnumeric():
            value="NUMBER"
        else:
            value=word.lower()
            if engDict.check(value):
                value=ps.stem(value)
            else:
                value="UNKNOWN"
        
        if(d.get(value) == None):
            U[value]=totalCount
            totalCount+=1
            d[value]=1
        else:
            d[value]=d[value]+1
            
    
    
    return d

def readData(PATH,spam,U,totalCount):
    dirList=os.listdir(PATH)
    size=len(dirList)
#     print(size)
    X=[]
    y=[spam for i in range(size)]
    for file in dirList:
        with open(PATH+'/'+file, 'r',errors='ignore') as f:
#             print(file)
            content=f.read()
            contentDict=preprocess(content,U,totalCount)
#         print(contentDict)
        X.append(contentDict)
    return X,y

def makeDataStructure(spamX,spamY,U):
    m=len(spamX)
    d=len(U)
    X=np.zeros([d,m])
    for i in range(m):
        for word in spamX[i]:
            if U.get(word) == None:
                continue
            X[U[word]][i]=spamX[i][word]

    return X
    
        

U={}   
totalCount=0
SPAM_PATH="Dataset/enron1/spam"
HAM_PATH="Dataset/enron1/ham"
spamX,spamY=readData(SPAM_PATH,1,U,totalCount)
hamX,hamY=readData(HAM_PATH,0,U,totalCount)
# print(U)

# print(X)
# listOfWords=list(U)


In [6]:
XSpam=makeDataStructure(spamX,spamY,U)
XHam=makeDataStructure(hamX,hamY,U)

# TRAINING

In [1]:
import numpy as np
import os
from nltk.stem import PorterStemmer
import nltk
import enchant
import pickle

In [2]:
def loadData():
    spamfile=open('spam', 'rb')
    hamfile=open('ham', 'rb')
    Ufile= open('u', 'rb')
    
    U=pickle.load(Ufile)
    XSpam=pickle.load(spamfile)
    XHam=pickle.load(hamfile)
    
    spamfile.close()
    hamfile.close()
    Ufile.close()
    
    return U,XSpam,XHam

U,XSpam,XHam=loadData()

In [3]:
def storeRepresentation(p,mean,cov,U):
    pfile = open('p', 'ab')
    meanfile = open('mean', 'ab')
    covfile = open('cov', 'ab')
    
    pickle.dump(p, pfile)
    pickle.dump(mean, meanfile)
    pickle.dump(cov, covfile)
    
    pfile.close()
    meanfile.close()
    covfile.close()
    
def loadReprsentation():
    pfile = open('p', 'rb')
    meanfile = open('mean', 'rb')
    covfile = open('cov', 'rb')
    Ufile= open('u', 'rb')
    
    p=pickle.load(pfile)
    mean=pickle.load(meanfile)
    cov=pickle.load(covfile)
    U=pickle.load(Ufile)
    
    pfile.close()
    meanfile.close()
    covfile.close()
    Ufile.close()
    
    return p,mean,cov,U

array([[ 1.,  2.,  1., ...,  1.,  1., 32.],
       [10., 24.,  1., ...,  5.,  1.,  1.],
       [ 1.,  1.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [1]:
def naiveBayesTraining(spamX,hamX):
    spamSize=spamX.shape[1]
    hamSize=hamX.shape[1]
    size=hamX.shape[1]+spamX.shape[1]
    mean=np.zeros([spamX.shape[0],2])
    data=np.concatenate((spamX,hamX),axis=1)
    p=spamSize/size
    mean[:,0]=np.sum(spamX,axis=1)/spamSize
    mean[:,1]=np.sum(hamX,axis=1)/hamSize
    cov=(data@data.transpose())/size
    
    
    return p,mean,cov



In [15]:
storeRepresentation(p,mean,cov,U)

# TESTING

In [2]:
import numpy as np
import os
from nltk.stem import PorterStemmer
import nltk
import enchant
import pickle

In [3]:
def gaussianPart(x,mean,cov):
    
    d=cov.shape[0]
    print(x.shape)
    print(mean.shape)
    mean=mean.reshape(d,1)
    sub=x-mean
    print(sub.shape)
    sub=sub.reshape(d,1)
    expTerm=-(sub@np.linalg.pinv(cov)@(sub.transpose()))
#     expTerm=-(sub@(sub.transpose()))
    return np.exp(expTerm)

def naiveBayesPrediction(x,p,mean,cov):
    spamProbability=gaussianPart(x,mean[:,1],cov)*p
    hamProbability=gaussianPart(x,mean[:,0],cov)*(1-p)
    if(spamProbability >= hamProbability):
        return 1
    else:
        return 0
    

In [ ]:
def loadReprsentation():
    pfile = open('p', 'rb')
    meanfile = open('mean', 'rb')
    covfile = open('cov', 'rb')
    Ufile= open('u', 'rb')
    
    p=pickle.load(pfile)
    mean=pickle.load(meanfile)
    cov=pickle.load(covfile)
    U=pickle.load(Ufile)
    
    pfile.close()
    meanfile.close()
    covfile.close()
    Ufile.close()
    
    return p,mean,cov,U

In [ ]:
def testPreprocess(content,U):
    words=content.split()
    ps = PorterStemmer()
    engDict=enchant.Dict("en_US")
    d=len(U)
    xTest=np.zeros([d,1])
    for word in words:
        if (not word.isalnum()) and (word != '$'):
            continue
        if word.isnumeric():
            value="NUMBER"
        else:
            value=word.lower()
            if engDict.check(value):
                value=ps.stem(value)
            else:
                value="UNKNOWN"
        
        if(U.get(value) == None):
            continue
            
        xTest[U[value]][0]+=1
                
    return xTest

def prediction(file,U,p,mean,cov):
    content=file.read()
    xTest=testPreprocess(content,U)
#     print(xTest.shape)
#     print(mean.shape)
    yPredicted=naiveBayesPrediction(xTest,p,mean,cov)
    print(yPredicted)

with open("Dataset/enron1/spam/0006.2003-12-18.GP.spam.txt") as f:
    p,mean,cov,U=loadReprsentation()
    prediction(f,U,p,mean,cov)
    

(14054, 1)
(14054,)
(14054, 1)
(14054, 1)
(14054,)
(14054, 1)


In [6]:
p,mean,cov=naiveBayesTraining(XSpam,XHam)
cov=np.dot(np.identity(cov.shape[0]),cov)

#testing folder
TEST_PATH_SPAM="Dataset/enron6/spam"
TEST_PATH_HAM="Dataset/enron6/ham"
p,mean,U=loadReprsentation()

def test(PATH,p,mean,cov,U,spam):
    
    dirList=os.listdir(PATH)
    size=len(dirList)
    count=0
    for file in dirList:
        with open(PATH+'/'+file, 'r',errors='ignore') as f:
                content=f.read()
                xTest=testPreprocess(content,U)
                yPredicted=naiveBayesPrediction(xTest,p,mean,cov)
                count+=yPredicted
    if spam == 1:
        print(count/size)
    else:
        print(1 - count/size)
                
test(TEST_PATH_SPAM,p,mean,cov,U,1)
test(TEST_PATH_HAM,p,mean,cov,U,1)      
    


SyntaxError: invalid syntax (3750141197.py, line 1)